In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['AUTOGRAPH_VERBOSITY'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.get_logger().setLevel('ERROR')
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Layer
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from math import sqrt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import h5py
import time
import warnings

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

pd.options.mode.chained_assignment = None
start_time = time.time()

# Google Drive'ı bağla
from google.colab import drive
drive.mount('/content/drive')


save_dir = '/content/drive/My Drive/colab_tez_icerikleri/Yeni'
os.makedirs(save_dir, exist_ok=True)

# HDF5 dosya adı
hdf5_filename = os.path.join(save_dir, 'model_ve_sonuclar_mevsimsel_momentumsuz.h5')

data = pd.read_excel('/content/drive/My Drive//hepsi2.xlsx')

# Tarih sütununu indeks olarak ayarlama ve sıralama
data['Sipariş Tarihi'] = pd.to_datetime(data['Sipariş Tarihi'])
data.set_index('Sipariş Tarihi', inplace=True)
data.sort_index(inplace=True)

# Günlük toplam satışları hesapla
daily_sales = data['Net fiyat'].resample('D').sum()

# Eğitim ve test verilerini ayırma
train_start_date = '2022-01-01'
train_end_date = '2023-05-31'
test_start_date = '2023-05-31'
test_end_date = '2023-10-31'

train_data = data[train_start_date:train_end_date]
test_data = data[test_start_date:test_end_date]

# Veriyi Ölçeklendirme
scaler = MinMaxScaler()
train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.transform(test_data)

# Veri Frekansını Belirleme
data_frequency = pd.infer_freq(data.index)
if data_frequency is None:
    data_frequency = 'D'

# Çoklu Yama Boyutu Projeksiyon Katmanları
patch_sizes = {
    'D': [7, 14],
}

selected_patch_sizes = patch_sizes.get(data_frequency, [16,64])
print(f"Seçilen yama boyutları: {selected_patch_sizes}")

def create_time_series_data(data, time_steps=50):
    X, y = [], []
    for i in range(time_steps, len(data)):
        X.append(data[i - time_steps:i, :])
        y.append(data[i, :])
    return np.array(X), np.array(y)

# Gerçek satış verileri
total_sales_2023 = test_data['Net fiyat'].resample('D').sum()

# Hata metriklerini kaydetmek için sözlük
metrics = {
    'r2': [],
    'mae': [],
    'rmse': [],
    'mape': []
}

# Tüm iterasyon sonuçlarını saklayacak liste
all_iterations = []

patch_size_results = {size: [] for size in selected_patch_sizes}

# HDF5 dosyası varsa yükle, yoksa modeli eğit ve kaydet
if os.path.exists(hdf5_filename):
    print("HDF5 dosyası bulundu, sonuçlar yükleniyor...")
    with h5py.File(hdf5_filename, 'r') as hdf:
        for iteration in range(10):
            iteration_group = hdf[f'iteration_{iteration + 1}']
            predicted_sales = iteration_group['predicted_sales'][:]
            real_sales_iteration = iteration_group['real_sales'][:]
            errors = iteration_group['errors'][:]

            # Hata metriklerini yükle
            metrics['r2'].append(iteration_group.attrs['r2'])
            metrics['mae'].append(iteration_group.attrs['mae'])
            metrics['rmse'].append(iteration_group.attrs['rmse'])
            metrics['mape'].append(iteration_group.attrs['mape'])

            # İterasyon sonuçlarını yükle
            iteration_results = {
                'iteration': iteration_group.attrs['iteration'],
                'r2': iteration_group.attrs['r2'],
                'mae': iteration_group.attrs['mae'],
                'rmse': iteration_group.attrs['rmse'],
                'mape': iteration_group.attrs['mape'],
                'süre (dakika)': iteration_group.attrs['süre (dakika)'],
                'süre (saniye)': iteration_group.attrs['süre (saniye)']
            }
            all_iterations.append(iteration_results)

            # LSTM loss history'yi yükle
            LSTM_loss_history = iteration_group.attrs.get('LSTM_loss_history', [])

            # Patch size sonuçlarını yükle
            for patch_size in selected_patch_sizes:
                if f'patch_size_{patch_size}' in iteration_group:
                    patch_group = iteration_group[f'patch_size_{patch_size}']
                    patch_size_results[patch_size].append({
                        'iteration': iteration + 1,
                        'r2': patch_group.attrs['r2'],
                        'mae': patch_group.attrs['mae'],
                        'rmse': patch_group.attrs['rmse'],
                        'mape': patch_group.attrs['mape']
                    })
else:
    print("HDF5 dosyası bulunamadı, model eğitiliyor ve sonuçlar kaydediliyor...")
    with h5py.File(hdf5_filename, 'w') as hdf:
        for iteration in range(10):
            iteration_start_time = time.time()
            print(f"\nIteration {iteration + 1}")

            LSTM_loss_history = []

            class LSTMLossHistory(tf.keras.callbacks.Callback):
                def on_epoch_end(self, epoch, logs={}):
                    LSTM_loss_history.append(logs.get('loss'))

            # Her patch size için ayrı model oluştur ve eğit
            models = []
            for patch_size in selected_patch_sizes:
                X_train, y_train = create_time_series_data(train_data_scaled, time_steps=patch_size)

                model = Sequential([
                    Dense(200, activation='relu', input_shape=(patch_size, X_train.shape[2])),  # Mevsimsel Momentum Katmanı kaldırıldı
                    LSTM(200, activation='relu', dropout=0.2, recurrent_dropout=0.2),
                    Dense(y_train.shape[1])
                ])
                model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
                model.fit(X_train, y_train, epochs=20, batch_size=64,
                          callbacks=[LSTMLossHistory()], verbose=0)
                models.append(model)

                # Her patch size için ayrı tahmin ve metrik hesaplama
                X_test, y_test = create_time_series_data(test_data_scaled, time_steps=patch_size)
                pred = model.predict(X_test)

                # Tahminleri ters ölçeklendirme
                pred_rescaled = scaler.inverse_transform(pred)
                y_test_rescaled = scaler.inverse_transform(y_test)

                # Performans metriklerini hesapla
                r2 = r2_score(y_test_rescaled, pred_rescaled)
                mae = mean_absolute_error(y_test_rescaled, pred_rescaled)
                rmse = sqrt(mean_squared_error(y_test_rescaled, pred_rescaled))
                mape = np.mean(np.abs((y_test_rescaled - pred_rescaled) / (y_test_rescaled + 1e-6))) * 100

                patch_size_results[patch_size].append({
                    'iteration': iteration + 1,
                    'r2': r2,
                    'mae': mae,
                    'rmse': rmse,
                    'mape': mape
                })

            # En kısa tahmin uzunluğunu bul
            predictions = [model.predict(create_time_series_data(test_data_scaled, time_steps=size)[0]) for size, model in zip(selected_patch_sizes, models)]
            min_length = min(len(p) for p in predictions)

            # Tüm tahminleri en kısa olana göre kırp
            predictions = [p[:min_length] for p in predictions]

            # Tahminlerin ortalamasını al
            LSTM_predicted = np.mean(predictions, axis=0)

            # Random Forest Modeli
            rf_model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
            rf_model.fit(LSTM_predicted, y_test[:min_length])  # y_test'i de kırp

            # Random Forest ile Düzeltilmiş Tahminler
            final_predicted = rf_model.predict(LSTM_predicted)
            final_predicted = scaler.inverse_transform(final_predicted)

            # Tahminleri bir DataFrame'e dönüştür
            predicted_df = pd.DataFrame(final_predicted, columns=test_data.columns)
            predicted_df.index = test_data.index[:len(predicted_df)]

            # Tahmin satış tutarlarını hesapla
            predicted_total_sales = predicted_df['Net fiyat'].resample('D').sum()

            # Gerçek satış verileri
            real_sales_iteration = total_sales_2023.values[:len(predicted_total_sales)]

            predicted_sales = predicted_total_sales.values

            # Performans metriklerini hesapla
            r2 = r2_score(real_sales_iteration, predicted_total_sales)
            mae = mean_absolute_error(real_sales_iteration, predicted_total_sales) / (
                        np.max(real_sales_iteration) - np.min(real_sales_iteration)) * 100
            rmse = sqrt(mean_squared_error(real_sales_iteration, predicted_total_sales)) / (
                        np.max(real_sales_iteration) - np.min(real_sales_iteration)) * 100

            epsilon = 1e-6
            mape = np.mean(
                np.abs((real_sales_iteration - predicted_total_sales) / (real_sales_iteration + epsilon))) * 100

            print("R-kare (R-squared) Değeri:", r2)
            print("Ortalama Hata (Mean Absolute Error):", mae)
            print("Kök Ortalama Kare Hata (RMSE):", rmse)
            print("Ortalama Mutlak Yüzde Hata (MAPE):", mape)

            # Hata metriklerini kaydet
            metrics['r2'].append(r2)
            metrics['mae'].append(mae)
            metrics['rmse'].append(rmse)
            metrics['mape'].append(mape)

            iteration_end_time = time.time()
            iteration_elapsed_time = iteration_end_time - iteration_start_time
            minutes = int(iteration_elapsed_time // 60)
            seconds = int(iteration_elapsed_time % 60)

            # Her iterasyon sonuçlarını kaydet
            iteration_results = {
                'iteration': iteration + 1,
                'r2': r2,
                'mae': mae,
                'rmse': rmse,
                'mape': mape,
                'süre (dakika)': minutes,
                'süre (saniye)': seconds,
            }
            all_iterations.append(iteration_results)

            # Hataları hesapla
            errors = real_sales_iteration - predicted_sales

            # İterasyon sonuçlarını HDF5 dosyasına kaydet
            iteration_group = hdf.create_group(f'iteration_{iteration + 1}')
            iteration_group.create_dataset('predicted_sales', data=predicted_sales)
            iteration_group.create_dataset('real_sales', data=real_sales_iteration)
            iteration_group.create_dataset('errors', data=errors)
            for key, value in iteration_results.items():
                iteration_group.attrs[key] = value

            # LSTM loss history'yi kaydet
            iteration_group.attrs['LSTM_loss_history'] = LSTM_loss_history

# 10 çalıştırmanın en iyi, en kötü ve ortalama skorlarını hesapla
eniyi_r2 = max(metrics['r2'])
eniyi_mae = min(metrics['mae'])
eniyi_rmse = min(metrics['rmse'])
eniyi_mape = min(metrics['mape'])

enkotu_r2 = min(metrics['r2'])
enkotu_mae = max(metrics['mae'])
enkotu_rmse = max(metrics['rmse'])
enkotu_mape = max(metrics['mape'])

ortalama_r2 = np.mean(metrics['r2'])
ortalama_mae = np.mean(metrics['mae'])
ortalama_rmse = np.mean(metrics['rmse'])
ortalama_mape = np.mean(metrics['mape'])

# En iyi, en kötü ve ortalama sonuçlar için bir DataFrame oluştur (devamı)
summary_results = {
    'eniyi_r2': [eniyi_r2],
    'eniyi_mae': [eniyi_mae],
    'eniyi_rmse': [eniyi_rmse],
    'eniyi_mape': [eniyi_mape],
    'enkotu_r2': [enkotu_r2],
    'enkotu_mae': [enkotu_mae],
    'enkotu_rmse': [enkotu_rmse],
    'enkotu_mape': [enkotu_mape],
    'ortalama_r2': [ortalama_r2],
    'ortalama_mae': [ortalama_mae],
    'ortalama_rmse': [ortalama_rmse],
    'ortalama_mape': [ortalama_mape]
}
summary_df = pd.DataFrame(summary_results)

iterations_df = pd.DataFrame(all_iterations)

# Excel dosyasına kaydet
with pd.ExcelWriter(os.path.join(save_dir, 'seasonal_momentumsuz_LSTM_rf_results.xlsx')) as writer:
    iterations_df.to_excel(writer, sheet_name='Iterations', index=False)
    summary_df.to_excel(writer, sheet_name='Summary', index=False)

# Grafikleri oluştur ve kaydet
def save_plt_figure(fig, filename):
    filepath = os.path.join(save_dir, filename)
    fig.savefig(filepath)
    plt.close(fig)

# 1. Gerçek vs Tahmin Satışlar Grafiği
plt.figure(figsize=(12, 6))
plt.plot(total_sales_2023.index[:len(predicted_sales)], real_sales_iteration, label='Gerçek Satışlar', color='blue')
plt.plot(total_sales_2023.index[:len(predicted_sales)], predicted_sales, label='Tahminler', color='red')
plt.title('Gerçek vs Tahmin Satışlar')
plt.xlabel('Tarih')
plt.ylabel('Net Fiyat')
plt.legend()
plt.tight_layout()
save_plt_figure(plt.gcf(), '1_gercek_vs_tahmin_satislar_mevsimsel_momentumsuz.png')

# 2. Performans Metrikleri Karşılaştırma Grafikleri (Ayrı ayrı)
metrics_list = ['R-squared', 'MAE', 'RMSE', 'MAPE']
metric_values = [r2, mae, rmse, mape]

plt.figure(figsize=(12, 6))
for i in range(len(metrics_list)):
    plt.subplot(2, 2, i + 1)
    plt.bar(metrics_list[i], metric_values[i])
    plt.title(metrics_list[i])
    plt.ylabel('Değer')
plt.tight_layout()
save_plt_figure(plt.gcf(), '2_performans_metrikleri_mevsimsel_momentumsuz.png')

# 3. Zaman İçinde Hata Değişimi Grafiği
plt.figure(figsize=(12, 6))
plt.plot(total_sales_2023.index[:len(errors)], errors)
plt.title('Tahmin Hatalarının Zaman İçindeki Değişimi')
plt.xlabel('Tarih')
plt.ylabel('Hata')
plt.tight_layout()
save_plt_figure(plt.gcf(), '3_zaman_icinde_hata_degisimi_mevsimsel_momentumsuz.png')

# 4. Eğitim Kaybı Grafiği
plt.figure(figsize=(12, 6))
plt.plot(LSTM_loss_history, label='Eğitim Kaybı')
plt.title('Eğitim Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
save_plt_figure(plt.gcf(), '4_egitim_kaybi_mevsimsel_momentumsuz.png')

# 5. Gerçek ve Tahmin Edilen Değerlerin Karşılaştırılması (Yoğunluk Grafiği)
plt.figure(figsize=(12, 6))
sns.kdeplot(real_sales_iteration, fill=True, label='Gerçek')
sns.kdeplot(predicted_sales, fill=True, label='Tahmin')
plt.title('Gerçek ve Tahmin Edilen Değerlerin Karşılaştırılması')
plt.xlabel('Satış Miktarı')
plt.ylabel('Yoğunluk')
plt.legend()
plt.tight_layout()
save_plt_figure(plt.gcf(), '5_gercek_vs_tahmin_yogunluk_mevsimsel_momentumsuz.png')

# 6. Günlük Satış Miktarlarının Zamana Göre Değişimi (Kutu Grafiği)
total_sales_df = pd.DataFrame(
    {'Tarih': total_sales_2023.index[:len(predicted_sales)], 'Satış Miktarı': real_sales_iteration})
total_sales_df['Haftanın Günü'] = total_sales_df['Tarih'].dt.dayofweek
plt.figure(figsize=(12, 6))
sns.boxplot(x='Haftanın Günü', y='Satış Miktarı', data=total_sales_df)
plt.title('Haftanın Günlerine Göre Satış Miktarları')
plt.xlabel('Gün')
plt.xticks(range(7), ['Pazartesi', 'Salı', 'Çarşamba', 'Perşembe', 'Cuma', 'Cumartesi', 'Pazar'])
plt.tight_layout()
save_plt_figure(plt.gcf(), '6_gunluk_satis_miktarlari_mevsimsel_momentumsuz.png')

# 7. Tahmin Hatalarının Otokorelasyon Fonksiyonu (ACF) ve Kısmi Otokorelasyon Fonksiyonu (PACF)
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig_acf = plot_acf(errors, lags=40)
save_plt_figure(fig_acf.figure, '7.1_hata_acf_mevsimsel_momentumsuz.png')

fig_pacf = plot_pacf(errors, lags=40)
save_plt_figure(fig_pacf.figure, '7.2_hata_pacf_mevsimsel_momentumsuz.png')

# 8. Mevsimsel Dekompozisyon Grafiği - Bu kısım kaldırıldı

# 9. Gerçek vs Tahmin Scatter Plot
plt.figure(figsize=(10, 10))
plt.scatter(real_sales_iteration, predicted_sales)
plt.plot([real_sales_iteration.min(), real_sales_iteration.max()],
         [real_sales_iteration.min(), real_sales_iteration.max()], 'r--')
plt.xlabel('Gerçek Değerler')
plt.ylabel('Tahmin Edilen Değerler')
plt.title('Gerçek vs Tahmin Scatter Plot')
save_plt_figure(plt.gcf(), '9_gercek_vs_tahmin_scatter_mevsimsel_momentumsuz.png')

# 10. Tahmin Doğruluğu Zaman Çizelgesi
plt.figure(figsize=(12, 6))
plt.plot(total_sales_2023.index[:len(predicted_sales)], np.abs(errors))
plt.title('Tahmin Doğruluğu Zaman Çizelgesi')
plt.xlabel('Tarih')
plt.ylabel('Mutlak Hata')
plt.tight_layout()
save_plt_figure(plt.gcf(), '10_tahmin_dogrulugu_zaman_cizelgesi_mevsimsel_momentumsuz.png')

# 11. Hata Dağılımı Isı Haritası
plt.figure(figsize=(12, 8))
error_df = pd.DataFrame({'Tarih': total_sales_2023.index[:len(errors)], 'Hata': errors})
error_df['Hafta'] = error_df['Tarih'].dt.to_period('W')
error_df['Gün'] = error_df['Tarih'].dt.dayofweek
pivot = error_df.pivot_table(values='Hata', index='Hafta', columns='Gün', aggfunc='mean')
sns.heatmap(pivot, cmap='coolwarm', center=0)
plt.title('Hata Dağılımı Isı Haritası')
plt.xlabel('Haftanın Günü')
plt.ylabel('Hafta')
plt.tight_layout()
save_plt_figure(plt.gcf(), '11_hata_dagilimi_isi_haritasi_mevsimsel_momentumsuz.png')

# 12. Kümülatif Satış Karşılaştırması
plt.figure(figsize=(12, 6))
plt.plot(total_sales_2023.index[:len(predicted_sales)], np.cumsum(real_sales_iteration), label='Gerçek Kümülatif Satışlar')
plt.plot(total_sales_2023.index[:len(predicted_sales)], np.cumsum(predicted_sales), label='Tahmin Edilen Kümülatif Satışlar')
plt.title('Kümülatif Satış Karşılaştırması')
plt.xlabel('Tarih')
plt.ylabel('Kümülatif Satış')
plt.legend()
plt.tight_layout()
save_plt_figure(plt.gcf(), '12_kumulatif_satis_karsilastirmasi_mevsimsel_momentumsuz.png')

# 13. Artık Değerlerin Q-Q Grafiği
from scipy import stats

plt.figure(figsize=(10, 10))
stats.probplot(errors, dist="norm", plot=plt)
plt.title("Artık Değerlerin Q-Q Grafiği")
plt.tight_layout()
save_plt_figure(plt.gcf(), '13_artik_degerlerin_qq_grafigi_mevsimsel_momentumsuz.png')

# 14. Tahmin Hatası vs Satış Miktarı Scatter Plot
plt.figure(figsize=(12, 8))
plt.scatter(real_sales_iteration, errors)
plt.title('Tahmin Hatası vs Satış Miktarı')
plt.xlabel('Gerçek Satış Miktarı')
plt.ylabel('Tahmin Hatası')
plt.tight_layout()
save_plt_figure(plt.gcf(), '14_tahmin_hatasi_vs_satis_miktari_mevsimsel_momentumsuz.png')

# 15. Dönemsel Performans Karşılaştırması
error_df['Ay'] = error_df['Tarih'].dt.to_period('M')
monthly_mae = error_df.groupby('Ay')['Hata'].apply(lambda x: np.mean(np.abs(x)))

plt.figure(figsize=(12, 6))
monthly_mae.plot(kind='bar')
plt.title('Aylık Ortalama Mutlak Hata (MAE)')
plt.xlabel('Ay')
plt.ylabel('MAE')
plt.xticks(rotation=45)
plt.tight_layout()
save_plt_figure(plt.gcf(), '15_aylik_mae_mevsimsel_momentumsuz.png')

# 16. Haftalık Satışı Görselleştirme - Bu kısım kaldırıldı

# 17. Tahmin Doğruluğu Yüzdesi Grafiği
plt.figure(figsize=(12, 6))
accuracy_percentage = (1 - np.abs(errors) / real_sales_iteration) * 100
plt.plot(total_sales_2023.index[:len(predicted_sales)], accuracy_percentage)
plt.title('Tahmin Doğruluğu Yüzdesi')
plt.xlabel('Tarih')
plt.ylabel('Doğruluk Yüzdesi')
plt.ylim(0, 100)
save_plt_figure(plt.gcf(), '17_tahmin_dogrulugu_yuzdesi_mevsimsel_momentumsuz.png')

# 18. Residual Lag Plot
plt.figure(figsize=(10, 10))
plt.scatter(errors[:-1], errors[1:])
plt.title('Residual Lag Plot')
plt.xlabel('Hata (t)')
plt.ylabel('Hata (t+1)')
save_plt_figure(plt.gcf(), '18_residual_lag_plot_mevsimsel_momentumsuz.png')

# 19. Hata Dağılımının Normallik Testi
from scipy import stats
_, p_value = stats.shapiro(errors)
plt.figure(figsize=(10, 6))
sns.histplot(errors, kde=True)
plt.title(f'Hata Dağılımı (Shapiro-Wilk p-value: {p_value:.4f})')
plt.xlabel('Hata')
plt.ylabel('Frekans')
save_plt_figure(plt.gcf(), '19_hata_dagilimi_normallik_mevsimsel_momentumsuz.png')

# 20. Tahmin Sapması Analiz Grafiği
bias = np.mean(predicted_sales - real_sales_iteration)
plt.figure(figsize=(10, 6))
plt.axhline(y=bias, color='r', linestyle='--')
plt.plot(total_sales_2023.index[:len(predicted_sales)], predicted_sales - real_sales_iteration)
plt.title(f'Tahmin Sapması (Ortalama Sapma: {bias:.2f})')
plt.xlabel('Tarih')
plt.ylabel('Tahmin - Gerçek Değer')
save_plt_figure(plt.gcf(), '20_tahmin_sapmasi_mevsimsel_momentumsuz.png')

# 21. Yüzdelik Hata Dağılım Grafiği
percentage_errors = (errors / real_sales_iteration) * 100
plt.figure(figsize=(10, 6))
sns.histplot(percentage_errors, kde=True)
plt.title('Yüzdelik Hata Dağılımı')
plt.xlabel('Yüzdelik Hata')
plt.ylabel('Frekans')
save_plt_figure(plt.gcf(), '21_yuzdelik_hata_dagilimi_mevsimsel_momentumsuz.png')

print("Tüm grafikler başarıyla kaydedildi.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Seçilen yama boyutları: [7, 14]
HDF5 dosyası bulunamadı, model eğitiliyor ve sonuçlar kaydediliyor...

Iteration 1
728/728 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
728/728 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
728/728 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
728/728 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
R-kare (R-squared) Değeri: 0.9920267921946482
Ortalama Hata (Mean Absolute Error): 0.7424621854738934
Kök Ortalama Kare Hata (RMSE): 1.1713507328730097
Ortalama Mutlak Yüzde Hata (MAPE): 9.96427656756276

Iteration 2
728/728 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
728/728 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
728/728 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
728/728 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
R-kare (R-squared) Değeri: 0.9913296030281885
Ortalama Hata (Mean Absolute Error): 0.733027981489346
Kök Ortalama Kare Hata (RMSE): 1.2214899637375998
Ortalama Mutlak Yüzde Hata (MAPE): 10.022910775655427

Itera